In [1]:
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import json


In [2]:
mne.set_log_level('ERROR')

In [3]:
class SeizureSense(nn.Module):
    
    def __init__(self):
        super(SeizureSense,self).__init__()
        
        #Block 1
        #should be taking in an input of 23x512
        #first layer temporal filters
        self.conv1=nn.Conv2d(1,8,(1,128),stride=1,padding=0)
        self.batchnorm1 = nn.BatchNorm2d(8, False)
        
        #spatial layer(depthwise layer)
        self.conv2=nn.Conv2d(8,32,(23,1))
        self.batchnorm2 = nn.BatchNorm2d(32, False)
        self.avgpool1 = nn.AvgPool2d((1,2))
        #apply dropout here in forward
        
        
        #Block 2
        #sepereable convolutional 2d
        self.conv3=nn.Conv2d(32,32,(1,16),stride=1)
        self.batchnorm3 = nn.BatchNorm2d(32,False) #CHECK THIS!
        self.avgpool2 = nn.AvgPool2d((1,16))

        
        #Block 3
        
        self.fc1= nn.Linear(96, 30)
        self.fc2=nn.Linear(30,3)

        #apply dropout here in forward
        self.dropout = nn.Dropout(0.25)
        
    def forward(self,x):
        x=self.conv1(x)
        x =F.elu(x)
        x=self.batchnorm1(x)
        x = self.dropout(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x=self.batchnorm2(x)
        x = self.dropout(x)
        x=self.avgpool1(x)
        
        x=F.elu(self.conv3(x))
        x=self.batchnorm3(x)
        x = self.dropout(x)
        x=self.avgpool2(x)
        
        #Fully connected time
        x=x.view(-1,96)
        x=F.elu(self.fc1(x))
        # FOR TRAINING NO NEED FOR SIGMOUD ON LAST THING BC CROSS ENTRTOPY LOSS SO ADD IT IN INFERENCE DONT FORGET
        x=self.fc2(x)
        return x

In [5]:
model=SeizureSense()
state_dict = torch.load(r"C:\Users\uddha\Downloads\SeizureSenseStateDict (1).pth",map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [9]:
#maps back the labels to words
def decode_predictions(predictions):
    label_names = ['interictal', 'preictal', 'ictal']
    _, predicted_labels = torch.max(predictions, 1)
    return [label_names[label] for label in predicted_labels]

In [ ]:
#loading data
raw = mne.io.read_raw_edf(r"C:\Users\uddha\Downloads\chb02_19.edf")
raw.load_data()
#Interictal time steps
#2000 seceonn-2001 seceonds
#Preictal 
#2779 seceonds-2780 seceonds
#inseceonds to 3371 seceonds

In [7]:
numpy_array=raw.get_data()
segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
print(segment_tensor.shape)

torch.Size([1, 1, 23, 921600])


In [10]:

#testing on different inputs: no preprocessing
input=segment_tensor[:,:,:,256*3370:256*3371]

output=model(input)
output=F.softmax(output,dim=1)
predicted_label_names = decode_predictions(output)
print(predicted_label_names)

['interictal']


In [11]:
#testing with preprocessing
#filtering 60 hz out with notch filtering

def preprocessing(raw):
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


    freqs = (60,120)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
    raw_notch.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    numpy_array=raw_notch.get_data()
    segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
    return segment_tensor

In [12]:
segment_tensor=preprocessing(raw)

In [20]:
input=segment_tensor[:,:,:,256*2779:256*2780]
output=model(input)
output=F.softmax(output,dim=1)
predicted_label_names = decode_predictions(output)
print(predicted_label_names)

['interictal']


In [33]:
#testung model1 
model1=SeizureSense()
#without preprocessing
input=segment_tensor[:,:,:,256*2000:256*2001]
output=model1(input)
predicted_label_names = decode_predictions(output)
print(predicted_label_names)


['interictal']


In [ ]:
CHB_files={'CHB-MIT/CHB_Database/chb02/chb02_19.edf': [(0, 2769, 'interictal'), (2769, 3369, 'preictal'), (3369, 3378, 'ictal'), (3378, 3600, 'interictal')],}
true_labels=[]
start,end=2778,3388
for i in range(start,end):
    for interval in CHB_files.values():
        for start_interval,end_interval,label in interval:
            if start_interval <= i < end_interval:
                true_labels.append(label)
                break
        else:
            continue
        break
    
true_labels = list(true_labels)
print(true_labels)

    

In [37]:
#loop within raw file
segment_tensor=preprocessing(raw)
predictions=[]
for i in range(2778,3388):
    input=segment_tensor[:,:,:,256*i:256*(i+1)]
    output=model1(input)
    output=F.softmax(output,dim=1)
    predicted_label_names=decode_predictions(output)
    predictions.append(predicted_label_names)
    


In [25]:
df = pd.DataFrame([predictions, true_labels])
csv_file_path = 'label_comparisons.csv'
df.to_csv(csv_file_path, index=False)

In [40]:
count = 0
for i in range(len(true_labels)):
    if true_labels[i] == predictions[i]:
        count += 1

print(count)

0


In [29]:

count = 0
for i in range(len(true_labels)):
    if true_labels[i] == predictions[i]:
        count += 1

print(count)


0
